# ARMIA and GARCH

Espen Sirnes  
2025-04-24

# Overview

The raw, non‑stationary residuals $\mathbf{u}$ are defined as:

<span id="eq-residuals">$$
\mathbf{u} = \mathbf{y} - \mathbf{X} \boldsymbol{\beta},
 \qquad(1)$$</span>

where $\mathbf{X}$ is an $T \times K$ array of $K$ covariates across $N$
groups and $T$ time periods, $\boldsymbol{\beta}$ is a $K \times 1$
coefficient vector, and $\mathbf{y}$ is the $T \times 1$ dependent
variable.

The reasons for handling this problems are

-   Serial correlation in $\mathbf u$ can understate standard errors,
    artificially inflating $t$–statistics.
-   Conditional heteroscedasticity violates the constant‑variance
    assumption.
-   Converting $\mathbf{u}$ into *i.i.d.* innovations permits valid
    inference.

The objective is therefore to transform $\mathbf{u}$ — typically
serially correlated and heteroscedastic — into uncorrelated,
homoscedastic residuals $\boldsymbol{\varepsilon}/\boldsymbol{\sigma}$
by applying

1.  **ARIMA filtering** removes mean persistence and unit‑roots,
    delivering the quasi‑innovations $\boldsymbol{\varepsilon}$.
2.  **GARCH standardisation** scales out time‑varying conditional
    variance, yielding the final innovations
    $\tilde{\varepsilon}_t = \varepsilon_t/\sigma_t$.

Formally, the ARIMA step (Box et al. 2015) is

<span id="eq-ARIMA">$$
\boldsymbol{\varepsilon} = \mathbf{MA}^{-1} \cdot \mathbf{AR} \cdot \boldsymbol{\Delta}_d \mathbf{u}
 \qquad(2)$$</span>

where $\mathbf{u}$ is the $T \times 1$ vector, differenced $d$ times by
$\boldsymbol{\Delta}_d$. $\mathbf{MA}$ and $\mathbf{AR}$ are
$T \times T$ moving‑average and autoregressive matrices.

Heteroscedasticity is subsequently corrected by the GARCH (Bollerslev
1986; Engle 1982) normalization:

<span id="eq-GARCH">$$
\mathbf{\sigma}^{2}
= \mathbf{GA}_{\sigma}^{-1}\,\mathbf{G} + \mathbf{GA}_{\sigma}^{-1}\,\mathbf{AR}_{\sigma}\boldsymbol{\varepsilon}^{2}
 \qquad(3)$$</span>

where $\boldsymbol{\varepsilon}^{2}$ and $\boldsymbol{\sigma}^{2}$ are
$T \times 1$ vectors. $\mathbf{GA}_\sigma$ and $\mathbf{AR}_\sigma$
embed the GARCH and ARCH lag polynomials.

The matrix $\mathbf{G}$ is a column vector $T \times 1$ with zeros,
except the first term is the initial variance.

Let us now look a bit closer on how these transformations are obtained.

# The ARIMA process

As noted, $\mathbf{u}$ is the $T \times 1$ matrix of residuals. It is
linked to the stationary residuals $\boldsymbol{\varepsilon}$ via an
ARIMA process:

<span id="eq-ARIMA_sum0">$$
\boldsymbol{\Delta}_d \mathbf{u} = \sum_{j=1}^{p} \rho_{j} \mathbf{L}^{j}  \boldsymbol{\Delta}_d \mathbf{u} +  \boldsymbol{\varepsilon} + \sum_{j=1}^{q} \lambda_{j} \mathbf{L}^{j}  \boldsymbol{\varepsilon}
 \qquad(4)$$</span>

where $\mathbf{L}$ is the lag operator matrix, defined as:

<span id="eq-L">$$
\mathbf{L} =
\begin{bmatrix}
0 &        &        &        &        \\
1 & 0      &        &        &        \\
0 & 1      & 0      &        &        \\
\vdots & \ddots & \ddots & \ddots &        \\
0 & \cdots & 0 & 1 & 0
\end{bmatrix}_{T \times T}
 \qquad(5)$$</span>

You can verify yourself that premultiplying a column vector by
$\mathbf{L}$ shifts its elements down by one period, inserting a zero in
the first position. In this way, $\mathbf{L}$ effectively applies a
one-period lag to the variable it is multiplied with.

Although <a href="#eq-ARIMA_sum0" class="quarto-xref">Equation 4</a> may
appear complex at first glance—suggesting that “everything depends on
everything”—it simplifies neatly when expressed using lag polynomials:
<span id="eq-ARIMA_sum">$$
\left( \mathbf{I} - \sum_{j=1}^{p} \rho_{j} \mathbf{L}^{j} \right) \boldsymbol{\Delta}_d \mathbf{u} = \left( \mathbf{I} + \sum_{j=1}^{q} \lambda_{j} \mathbf{L}^{j} \right) \boldsymbol{\varepsilon}
 \qquad(6)$$</span>

By compacting the notation and defining the lag polynomials as matrices,
the expression becomes even clearer and more concise. The moving-average
(MA) component can then be written as:

<span id="eq-MA">$$
\mathbf{MA} = \mathbf{I} + \sum_{j=1}^{q} \lambda_{j} \mathbf{L}^{j},
 \qquad(7)$$</span>

and the corresponding autoregressive (AR) term is

<span id="eq-AR">$$
\mathbf{AR} = \mathbf{I} - \sum_{j=1}^{p} \rho_{j} \mathbf{L}^{j}.
 \qquad(8)$$</span>

The MA matrix typically look like this

<span id="eq-MA_exp">$$
\mathbf{MA} =
\begin{bmatrix}
1      &        &        &        &        &        \\
\lambda_1 & 1      &        &        &        &        \\
\lambda_2 & \lambda_1 & 1      &        &        &        \\
\vdots & \vdots & \ddots & \ddots &        &        \\
\lambda_q & \cdots & \cdots & \lambda_1 & 1      &        \\
0      & \ddots & \ddots & \ddots & \ddots & \ddots
\end{bmatrix}
 \qquad(9)$$</span>

And the AR matrix typically like this <span id="eq-AR_exp">$$
\mathbf{AR} =
\begin{bmatrix}
1      &        &        &        &        &        \\
-\rho_1 & 1      &        &        &        &        \\
-\rho_2 & -\rho_1 & 1      &        &        &        \\
\vdots & \vdots & \ddots & \ddots &        &        \\
-\rho_p & \cdots & \cdots & -\rho_1 & 1      &        \\
0      & \ddots & \ddots & \ddots & \ddots & \ddots
\end{bmatrix}
 \qquad(10)$$</span>

Hence, we can write
(<a href="#eq-ARIMA_sum" class="quarto-xref">Equation 6</a>) in matrix
notation as:

<span id="eq-ARIMA_matrix">$$
\mathbf{AR} \cdot \boldsymbol{\Delta}_d \mathbf{u} = \mathbf{MA} \cdot \boldsymbol{\varepsilon}
 \qquad(11)$$</span>

Our goal is to isolate the independent random noise embedded in the
differenced residuals $\boldsymbol{\Delta}_d\mathbf{u}$. Recognising
that this is a simple linear algebra problem, we can solve for the
white-noise sequence $\boldsymbol{\varepsilon}$ by premultiplying both
sides by the inverse of the moving-average operator $\mathbf{MA}^{-1}$:

<span id="eq-ARIMA_sol">$$
\boldsymbol{\varepsilon} = \mathbf{MA}^{-1} \cdot \mathbf{AR} \cdot \boldsymbol{\Delta}_d \mathbf{u}
 \qquad(12)$$</span>

Hence, the white noise error terms $\boldsymbol{\varepsilon}$ can be
obtained through standard linear algebra by inverting the moving average
operator. However, in practice, explicitly computing the inverse is
often computationally expensive, so a recursive filtering approach is
typically preferred for efficiency.

# The GARCH process

The primary objective of the GARCH transformation is to correct for
serially conditioned heteroskedasticity — that is, time-varying
volatility in the error process. Conceptually, the procedure closely
parallels the ARIMA approach, but with an inverted goal: whereas ARIMA
seeks to eliminate temporal dependence in the residuals by extracting
the *innovations*, GARCH focuses on modelling the *volatility* itself.

In this framework, we estimate the serially correlated conditional
standard deviation vector $\boldsymbol{\sigma}$, which is then used to
standardise the stationary ARIMA residuals $\boldsymbol{\varepsilon}$,
producing homoscedastic white noise via:

<span id="eq-objective">$$
\tilde{\varepsilon}_t = \varepsilon_t / \sigma_t
 \qquad(13)$$</span>

This is conceptually the inverse of the ARIMA transformation, which
seeks to *remove* serial correlation from the residuals. GARCH, by
contrast, models that correlation in volatility.

We begin with a general GARCH formulation. GARCH simply means that the
current variance depends on the past variance and the previous squared
residuals. This can be written as

<span id="eq-GARCH_problem">$$
\boldsymbol{\sigma}^{2} = \mathbf{G} + \sum_{j = 1}^{m} \psi_{j}\mathbf{L}^{j}\boldsymbol{\varepsilon}^2 + \sum_{j = 1}^{k} \gamma_{j} \mathbf{L}^{j} \boldsymbol{\sigma}^{2}
 \qquad(14)$$</span>

The matrix $\mathbf{G}$ is a column vector $T \times 1$ with zeros,
except the first term is the initial variance. As with ARIMA, we can
rearrange the GARCH equation into an explicit solvable form:

<span id="eq-GARCH_solvable">$$
\left( \mathbf{I} - \sum_{j = 1}^{k} \gamma_{j} \mathbf{L}^{j} \right) \boldsymbol{\sigma}^{2} = \mathbf{G} + \sum_{j = 1}^{m} \psi_{j} \mathbf{L}^{j} h\left( \boldsymbol{\varepsilon}, c \right)
 \qquad(15)$$</span>

As with the ARIMA model, the problem becomes more transparent when we
express the lag polynomials as matrix operators:

<span id="eq-GAs">$$
\mathbf{GA}_{\sigma} = \mathbf{I} - \sum_{j = 1}^{k} \gamma_{j} \mathbf{L}^{j}
 \qquad(16)$$</span>

<span id="eq-ARs">$$
\mathbf{AR}_{\sigma} = \sum_{j = 1}^{m} \psi_{j} \mathbf{L}^{j}
 \qquad(17)$$</span>

$\mathbf{GA}_{\sigma}$ and $\mathbf{AR}_{\sigma}$ have similar
structures as <a href="#eq-MA_exp" class="quarto-xref">Equation 9</a>
and <a href="#eq-AR_exp" class="quarto-xref">Equation 10</a>.

Then, analogous to the ARIMA formulation, we can express the
<a href="#eq-GARCH_solvable" class="quarto-xref">Equation 15</a> in
operator form as:

<span id="eq-GARCH_operator">$$
\mathbf{GA}_{\sigma} \cdot \boldsymbol{\sigma}^{2} =  \mathbf{G} + \mathbf{AR}_{\sigma} \cdot \boldsymbol{\varepsilon}^2
 \qquad(18)$$</span>

As is standard with linear systems, we solve for $\boldsymbol{\sigma}^2$
by premultiplying both sides by the inverse of $\mathbf{GA}_{\sigma}$:

<span id="eq-GARCH_solution">$$
\boldsymbol{\sigma}^{2} = \mathbf{GA}_{\sigma}^{-1} \left( \mathbf{G} + \mathbf{AR}_{\sigma} \cdot \boldsymbol{\varepsilon}^2 \right)
 \qquad(19)$$</span>

This transformation deflates the residuals by their conditional standard
deviation, yielding a series $\varepsilon_t / \sigma_t$ that is
homoscedastic under the model.

### Confusing terminology

Following convention, the effects from $\boldsymbol{\varepsilon}^2$ are
referred to as **autoregressive**, even though they represent the
response to past innovations. Historically, the term GARCH was
introduced by (Bollerslev 1986) to generalise the ARCH process by adding
a persistent variance component.

This naming choice may appear counterintuitive when compared to ARIMA
models, where:

-   **Autoregressive (AR)** components capture persistence over time.
-   **Moving average (MA)** components capture transient innovation
    shocks.

In GARCH models, however, the roles are reversed:

-   **ARCH terms** (lagged squared residuals) reflect immediate
    volatility responses — akin to MA terms in ARIMA.
-   **GARCH terms** (lagged variances) model long-run volatility
    persistence — functionally closer to AR components.

Thus, it might have been more consistent to label the ARCH component as
the moving average part. One possible justification for the current
convention is that the ARCH effect is the component inverted when
solving the reduced-form variance equation. However, it is crucial to
recognise that the dependent variable differs: ARIMA models solve for
the signal $\mathbf{y}$ (via $\boldsymbol{\varepsilon}$), while GARCH
models solve directly for the conditional variance
$\boldsymbol{\sigma}^{2}$.

# Literature

Bollerslev, Tim. 1986. “Generalized Autoregressive Conditional
Heteroskedasticity.” *Journal of Econometrics* 31 (3): 307–27.

Box, George EP, Gwilym M Jenkins, Gregory C Reinsel, and Greta M Ljung.
2015. *Time Series Analysis: Forecasting and Control*. John Wiley &
Sons.

Engle, Robert F. 1982. “Autoregressive Conditional Heteroscedasticity
with Estimates of the Variance of United Kingdom Inflation.”
*Econometrica: Journal of the Econometric Society*, 987–1007.